In [1]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *
# from rsna_retro.train3d import *
from rsna_retro.train_adjacent import *

Loading imports


In [2]:
torch.cuda.set_device(1)

## Train

In [3]:
name = 'train_adjacent_3c'

In [5]:
# dbunch = get_data_gen(Meta.df_any, Meta.grps_any, 512, sz=128, c3=True)
dbunch = get_adj_data(512, 128, Meta.splits_stg1, c3=True)
learn = get_learner(dbunch, xresnet18)

In [5]:
do_fit(learn, 20, 4e-2)
learn.save(f'runs/{name}-1')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.269365,0.284156,0.891961,0.782228,05:49
1,0.240021,0.252687,0.907981,0.831122,05:43
2,0.222274,0.274895,0.898848,0.789114,05:44
3,0.212675,0.256419,0.903037,0.817004,05:46
4,0.204610,0.208262,0.922973,0.863965,05:50
5,0.199007,0.209916,0.920333,0.856814,05:51
6,0.196785,0.215926,0.923330,0.843253,05:50
7,0.190333,0.199025,0.928972,0.866190,05:44
8,0.184499,0.217785,0.925767,0.854059,05:55
9,0.181565,0.185740,0.933417,0.879115,05:46


In [6]:
learn.dbunch = get_data_gen(Meta.df_any, Meta.grps_any, 512, sz=256, c3=True)
do_fit(learn, 20, 4e-3)
learn.save(f'runs/{name}-2')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.178314,0.190167,0.932243,0.876016,06:33
1,0.163995,0.176704,0.936936,0.886532,06:29
2,0.156380,0.172639,0.938507,0.888598,06:28
3,0.150417,0.172572,0.938715,0.887379,06:30
4,0.148932,0.167169,0.940829,0.893127,06:29
5,0.146415,0.170050,0.940185,0.888942,06:29
6,0.142422,0.169537,0.941231,0.892412,06:29
7,0.140100,0.164451,0.942140,0.895484,06:31
8,0.135010,0.166667,0.942630,0.896332,06:29
9,0.132576,0.170250,0.941760,0.895113,06:29


In [7]:
learn.dbunch = get_data_gen(Meta.df_comb, Meta.grps, 512, sz=None, c3=True)
do_fit(learn, 2, 5e-4)
learn.save(f'runs/{name}-3-full')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.097783,0.187456,0.941306,0.892915,06:17
1,0.093887,0.191735,0.941769,0.893312,06:16


## Submission

In [7]:
learn.load(f'runs/{name}-3-full')
sub_fn = f'subm/{name}'

In [8]:
# learn.dbunch = get_data_gen(Meta.df_tst, bs=512, sz=None, with_aug=False, tst_dir='tst_jpg256')
learn.dbunch = get_adj_test_data(512, sz=None, tst_dir='tst_jpg256', c3=True)

In [9]:
preds,targs = learn.get_preds()

In [10]:
pred_csv = submission(Meta.df_tst, preds, fn=sub_fn)

In [11]:
api.competition_submit(f'{sub_fn}.csv', f'{name} adjacent 3c', 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:03<00:00, 8.99MB/s]


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [17]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 14244564,
 'totalBytes': 27277209,
 'date': '2020-01-21T07:12:15.983Z',
 'description': 'train_adjacent_3c adjacent 3c',
 'errorDescription': None,
 'fileName': 'train_adjacent_3c.csv',
 'publicScore': '0.32314',
 'privateScore': '0.07443',
 'status': 'complete',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/14244564/14244564.raw'}